In [ ]:
from src.dataset.FaceLandmarksDataset import FaceLandmarksDataset
from src.model.FaceLandmarksModel import FacialLandmarksModel


In [ ]:
# Khởi tạo model đã train sẵn
path = '' # kh up được model lên github nên đẻ path rỗng up sau :v
model = FacialLandmarksModel.load_from_checkpoint(path)

In [ ]:
import matplotlib.pyplot as plt

# Hàm vẽ loss sau khi train trên toàn bộ tập dữ liệu 
def plot_loss(model):
    loss_values = [loss.cpu().item() for loss in model.loss_value] # Chuyển loss_value sang CPU nếu nó đang ở trên GPU
    plt.plot(loss_values, label='Validation Loss')
    plt.title('Loss per Validation Step')
    plt.xlabel('Validation Step')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_loss(model)


In [ ]:
# Detect trên một ảnh ngẫu nhiên thuộc tập Test
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
model.eval() 
dataset = FaceLandmarksDataset('/kaggle/input/ibug-300w/ibug_300W_large_face_landmark_dataset/labels_ibug_300W_test.xml')
index =  104 
image_path = dataset.image_filenames[index]

# Đọc ảnh và xử lí sao cho khớp với ảnh đầu vào model 
image = cv2.imread(image_path)
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


bbox = dataset.crops[index]
landmarks = dataset.landmarks[index]
x1 = int(bbox['left'])
y1 = int(bbox['top'])
x2 = int(bbox['left']) + int(bbox['width'])
y2 = int(bbox['top']) + int(bbox['height'])

cropped_image = image_gray[y1:y2, x1:x2]
resized_image = cv2.resize(cropped_image, (224, 224))
input_image = torch.tensor(resized_image, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0  # Thêm batch size và kênh

# Detect với model 
with torch.no_grad():
    predicted_landmarks = model(input_image)


predicted_landmarks = predicted_landmarks.view(-1, 2).numpy()
scale_x = cropped_image.shape[1] / 224.0
scale_y = cropped_image.shape[0] / 224.0
predicted_landmarks[:, 0] *= scale_x
predicted_landmarks[:, 1] *= scale_y


for (x, y) in predicted_landmarks:
    cv2.circle(cropped_image, (int(x), int(y)), 2, (255, 0, 0), -1)

plt.imshow(cropped_image, cmap='gray')
plt.show()


In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


model.eval() 
dataset = FaceLandmarksDataset('/kaggle/input/ibug-300w/ibug_300W_large_face_landmark_dataset/labels_ibug_300W_test.xml')

# Lưu trữ MES cho từng ảnh 
mse_per_image_list = []
sum_mse = 0 
# Vòng lặp qua tập test để dự đoán và tính MSE
with torch.no_grad():
    for idx in range(len(dataset)):
        # Lấy từng ảnh và nhãn từ test_dataset
        image, true_landmarks = dataset[idx]  # Lấy ảnh và nhãn
        image = image.unsqueeze(0)  # Thêm chiều batch vào ảnh

        predicted_landmarks = model(image)  # Dự đoán landmarks

        # Tính MSE cho ảnh hiện tại
        mse = F.mse_loss(predicted_landmarks, true_landmarks.unsqueeze(0), reduction='mean')  # Tính MSE cho từng ảnh
        sum_mse += mse 
        mse_per_image_list.append(mse.item())  # Thêm MSE vào danh sách

# In MSE trung bình trên tập dataset
print(mse/len(dataset))
# In MSE cho từng ảnh
for idx, mse in enumerate(mse_per_image_list):
    print(f'Ảnh {idx + 1}: MSE = {mse}')
